<a href="https://colab.research.google.com/github/Marcin19721205/Wprowadzenie-Do-Sieci-Neuronowych/blob/main/16_Keras08_housing_prices_regression02MJ.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Regresja - NN sekwencyjna z regularyzacją i dropout, zapisujaca wyniki tuningu




### <a name='a0'></a> 1. Import biblotek

In [1]:
%tensorflow_version 2.x
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import gc #garbage collector - usuwa dane, co do których nie ma odniesien w kodzie
import tensorflow as tf
import sklearn.datasets as skds
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

np.set_printoptions(precision=12, suppress=True, linewidth=150)
pd.options.display.float_format = '{:.6f}'.format
tf.__version__

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


'2.19.0'

### <a name='a1'></a> 2. Załadowanie danych i wstępna eksploracja

Zapis do dataset_raw  

In [2]:
raw_dataset = pd.read_csv('https://storage.googleapis.com/esmartdata-courses-files/ann-course/housing.csv')
raw_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   longitude           20640 non-null  float64
 1   latitude            20640 non-null  float64
 2   housing_median_age  20640 non-null  float64
 3   total_rooms         20640 non-null  float64
 4   total_bedrooms      20433 non-null  float64
 5   population          20640 non-null  float64
 6   households          20640 non-null  float64
 7   median_income       20640 non-null  float64
 8   median_house_value  20640 non-null  float64
 9   ocean_proximity     20640 non-null  object 
dtypes: float64(9), object(1)
memory usage: 1.6+ MB


Kopia danych surowych do dataset uzywany w kodzie

In [3]:
dataset = raw_dataset.copy()
dataset.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.230000,37.880000,41.000000,880.000000,129.000000,322.000000,126.000000,8.325200,452600.000000,NEAR BAY
1,-122.220000,37.860000,21.000000,7099.000000,1106.000000,2401.000000,1138.000000,8.301400,358500.000000,NEAR BAY
2,-122.240000,37.850000,52.000000,1467.000000,190.000000,496.000000,177.000000,7.257400,352100.000000,NEAR BAY
3,-122.250000,37.850000,52.000000,1274.000000,235.000000,558.000000,219.000000,5.643100,341300.000000,NEAR BAY
4,-122.250000,37.850000,52.000000,1627.000000,280.000000,565.000000,259.000000,3.846200,342200.000000,NEAR BAY


In [4]:
dataset.tail()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
20635,-121.090000,39.480000,25.000000,1665.000000,374.000000,845.000000,330.000000,1.560300,78100.000000,INLAND
20636,-121.210000,39.490000,18.000000,697.000000,150.000000,356.000000,114.000000,2.556800,77100.000000,INLAND
20637,-121.220000,39.430000,17.000000,2254.000000,485.000000,1007.000000,433.000000,1.700000,92300.000000,INLAND
20638,-121.320000,39.430000,18.000000,1860.000000,409.000000,741.000000,349.000000,1.867200,84700.000000,INLAND
20639,-121.240000,39.370000,16.000000,2785.000000,616.000000,1387.000000,530.000000,2.388600,89400.000000,INLAND


Sprawdzenie brakujących wartości

In [5]:
dataset.isnull().sum() / len(dataset)

,0
longitude,0.000000
latitude,0.000000
housing_median_age,0.000000
total_rooms,0.000000
total_bedrooms,0.010029
population,0.000000
households,0.000000
median_income,0.000000
median_house_value,0.000000
ocean_proximity,0.000000


Usuniecie rekordów z brakami

In [6]:
dataset.dropna(inplace=True) #usun rekordy, gdzie są NaN

dataset.isnull().sum() / len(dataset)

,0
longitude,0.000000
latitude,0.000000
housing_median_age,0.000000
total_rooms,0.000000
total_bedrooms,0.000000
population,0.000000
households,0.000000
median_income,0.000000
median_house_value,0.000000
ocean_proximity,0.000000


Statystyki dataset - numeryczne

In [7]:
dataset.describe().T

,count,mean,std,min,25%,50%,75%,max
longitude,20433.000000,-119.570689,2.003578,-124.350000,-121.800000,-118.490000,-118.010000,-114.310000
latitude,20433.000000,35.633221,2.136348,32.540000,33.930000,34.260000,37.720000,41.950000
housing_median_age,20433.000000,28.633094,12.591805,1.000000,18.000000,29.000000,37.000000,52.000000
total_rooms,20433.000000,2636.504233,2185.269567,2.000000,1450.000000,2127.000000,3143.000000,39320.000000
total_bedrooms,20433.000000,537.870553,421.385070,1.000000,296.000000,435.000000,647.000000,6445.000000
population,20433.000000,1424.946949,1133.208490,3.000000,787.000000,1166.000000,1722.000000,35682.000000
households,20433.000000,499.433465,382.299226,1.000000,280.000000,409.000000,604.000000,6082.000000
median_income,20433.000000,3.871162,1.899291,0.499900,2.563700,3.536500,4.744000,15.000100
median_house_value,20433.000000,206864.413155,115435.667099,14999.000000,119500.000000,179700.000000,264700.000000,500001.000000


Statystyki dataset - obiektowe

In [8]:
dataset.describe(include=['object']).T

,count,unique,top,freq
ocean_proximity,20433,5,<1H OCEAN,9034


Oblicz ilość wystąpień w klasie 'category'

In [9]:
dataset.ocean_proximity.value_counts()

,count
ocean_proximity,
<1H OCEAN,9034
INLAND,6496
NEAR OCEAN,2628
NEAR BAY,2270
ISLAND,5


Histogram Zmiennej Docelowej

In [10]:
px.histogram(dataset, x='median_house_value')
#px.histogram(dataset, x='median_income') #skośny

Ilość wystąpień zmiennej docelowej

In [11]:
dataset.median_house_value.value_counts()

,count
median_house_value,
500001.000000,958
137500.000000,119
162500.000000,116
112500.000000,103
187500.000000,92
...,...
397400.000000,1
492000.000000,1
476300.000000,1


Wartość 500001 jest niewiarygodna, jest to ucięta do tej wartości cena z inneg przedziału.

odrzucamy te wystąpienia

In [12]:
index_to_drop = dataset[dataset.median_house_value == 500001].index
dataset = dataset.drop(index=index_to_drop)
px.histogram(dataset, x='median_house_value')

Get dummies - do zakodowania danych kategorycznych, z drop first = True (pomin pierwsze)

In [13]:
dataset_dummies = pd.get_dummies(dataset, drop_first=True)
dataset_dummies = dataset_dummies.astype({col: float for col in dataset_dummies.select_dtypes('bool').columns}) #bez tego bool są true/false a teraz 0 i 1 - przy standaryzacji nie będzie błedów

dataset_dummies.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity_INLAND,ocean_proximity_ISLAND,ocean_proximity_NEAR BAY,ocean_proximity_NEAR OCEAN
0,-122.230000,37.880000,41.000000,880.000000,129.000000,322.000000,126.000000,8.325200,452600.000000,0.000000,0.000000,1.000000,0.000000
1,-122.220000,37.860000,21.000000,7099.000000,1106.000000,2401.000000,1138.000000,8.301400,358500.000000,0.000000,0.000000,1.000000,0.000000
2,-122.240000,37.850000,52.000000,1467.000000,190.000000,496.000000,177.000000,7.257400,352100.000000,0.000000,0.000000,1.000000,0.000000
3,-122.250000,37.850000,52.000000,1274.000000,235.000000,558.000000,219.000000,5.643100,341300.000000,0.000000,0.000000,1.000000,0.000000
4,-122.250000,37.850000,52.000000,1627.000000,280.000000,565.000000,259.000000,3.846200,342200.000000,0.000000,0.000000,1.000000,0.000000


### <a name='a2'></a> 3. Podział na zbiór treningowy oraz testowy

In [14]:
train_dataset = dataset_dummies.sample(frac=0.8, random_state=0)
test_dataset = dataset_dummies.drop(train_dataset.index)

print(f'train_dataset length: {len(train_dataset)}')
print(f'test_dataset length: {len(test_dataset)}')

train_dataset length: 15580
test_dataset length: 3895


In [15]:
train_dataset.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity_INLAND,ocean_proximity_ISLAND,ocean_proximity_NEAR BAY,ocean_proximity_NEAR OCEAN
19234,-122.690000,38.510000,18.000000,3364.000000,501.000000,1442.000000,506.000000,6.685400,313000.000000,0.000000,0.000000,0.000000,0.000000
1859,-124.140000,41.950000,21.000000,2696.000000,578.000000,1208.000000,494.000000,2.275000,122400.000000,0.000000,0.000000,0.000000,1.000000
51,-122.270000,37.820000,43.000000,1868.000000,456.000000,1061.000000,407.000000,1.504500,93800.000000,0.000000,0.000000,1.000000,0.000000
11192,-117.930000,33.820000,28.000000,2444.000000,555.000000,1848.000000,567.000000,3.017900,198800.000000,0.000000,0.000000,0.000000,0.000000
20355,-118.960000,34.190000,16.000000,1807.000000,346.000000,587.000000,296.000000,1.981100,162500.000000,0.000000,0.000000,0.000000,0.000000


In [16]:
test_dataset.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity_INLAND,ocean_proximity_ISLAND,ocean_proximity_NEAR BAY,ocean_proximity_NEAR OCEAN
10,-122.260000,37.850000,52.000000,2202.000000,434.000000,910.000000,402.000000,3.203100,281500.000000,0.000000,0.000000,1.000000,0.000000
13,-122.260000,37.840000,52.000000,696.000000,191.000000,345.000000,174.000000,2.673600,191300.000000,0.000000,0.000000,1.000000,0.000000
19,-122.270000,37.840000,52.000000,1503.000000,298.000000,690.000000,275.000000,2.603300,162900.000000,0.000000,0.000000,1.000000,0.000000
26,-122.280000,37.850000,49.000000,1130.000000,244.000000,607.000000,239.000000,2.459700,93800.000000,0.000000,0.000000,1.000000,0.000000
40,-122.260000,37.830000,52.000000,1665.000000,419.000000,946.000000,395.000000,2.097800,155400.000000,0.000000,0.000000,1.000000,0.000000


Relacje pomiedzy poszczególnymi zmiennymi

In [17]:
px.scatter_matrix(train_dataset, dimensions=['median_house_value', 'housing_median_age', 'median_income', 'total_rooms'], color='median_house_value', height=700)

Staystyki zbioru treningowego do standaryzacji

In [18]:
train_stats = train_dataset.describe() #wpisuje do train_stat statystyki opisowe train_dataset.describe()
train_stats.pop('median_house_value') #usuwam label ze statystyk
train_stats = train_stats.transpose() #przedstawiam jako transponowane
train_stats

,count,mean,std,min,25%,50%,75%,max
longitude,15580.000000,-119.558780,2.006237,-124.350000,-121.750000,-118.490000,-117.990000,-114.470000
latitude,15580.000000,35.648614,2.147016,32.550000,33.930000,34.270000,37.730000,41.950000
housing_median_age,15580.000000,28.489217,12.505895,1.000000,18.000000,29.000000,37.000000,52.000000
total_rooms,15580.000000,2620.014506,2195.654212,2.000000,1441.000000,2112.000000,3119.000000,39320.000000
total_bedrooms,15580.000000,539.900578,424.499548,2.000000,299.000000,436.000000,647.000000,6445.000000
population,15580.000000,1441.193068,1160.542775,3.000000,801.000000,1179.000000,1746.000000,35682.000000
households,15580.000000,501.099487,385.039835,2.000000,283.000000,411.000000,605.000000,6082.000000
median_income,15580.000000,3.672427,1.570297,0.499900,2.519975,3.440500,4.581425,15.000100
ocean_proximity_INLAND,15580.000000,0.333569,0.471503,0.000000,0.000000,0.000000,1.000000,1.000000
ocean_proximity_ISLAND,15580.000000,0.000193,0.013876,0.000000,0.000000,0.000000,0.000000,1.000000


#Wyrywamy labele - targety z danych

In [19]:
train_labels = train_dataset.pop('median_house_value') #wez labele train dataset
test_labels = test_dataset.pop('median_house_value') #as above

### <a name='a3'></a> 4. Standaryzacja danych
# średnia i Sx są liczone na zbiorze treningowym, ale stosowane na treningowym i testowym

Normalizacja danych

In [20]:
def norm(x):
    return (x - train_stats['mean']) / train_stats['std']

In [22]:
normed_train_data = norm(train_dataset) #znormalizuj dane
normed_test_data = norm(test_dataset) #znormalizuj dane

In [23]:
normed_train_data.isnull().sum()

,0
longitude,0
latitude,0
housing_median_age,0
total_rooms,0
total_bedrooms,0
population,0
households,0
median_income,0
ocean_proximity_INLAND,0
ocean_proximity_ISLAND,0


In [24]:
normed_train_data.head(3)

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity_INLAND,ocean_proximity_ISLAND,ocean_proximity_NEAR BAY,ocean_proximity_NEAR OCEAN
19234,-1.560743,1.332727,-0.838742,0.338845,-0.091639,0.000695,0.012727,1.918729,-0.707459,-0.013877,-0.346133,-0.378230
1859,-2.283490,2.934950,-0.598855,0.034607,0.089751,-0.200934,-0.018438,-0.889913,-0.707459,-0.013877,-0.346133,2.643727
51,-1.351396,1.011351,1.160315,-0.342501,-0.197646,-0.327599,-0.244389,-1.380585,-0.707459,-0.013877,2.888874,-0.378230


In [25]:
normed_test_data.head(3)

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity_INLAND,ocean_proximity_ISLAND,ocean_proximity_NEAR BAY,ocean_proximity_NEAR OCEAN
10,-1.346412,1.025324,1.879976,-0.190383,-0.249472,-0.457711,-0.257375,-0.298878,-0.707459,-0.013877,2.888874,-0.378230
13,-1.346412,1.020666,1.879976,-0.876283,-0.821910,-0.944552,-0.849521,-0.636075,-0.707459,-0.013877,2.888874,-0.378230
19,-1.351396,1.020666,1.879976,-0.508739,-0.569849,-0.647277,-0.587211,-0.680844,-0.707459,-0.013877,2.888874,-0.378230


To wynika z Numpy

In [26]:
normed_test_data = normed_test_data.values
normed_train_data = normed_train_data.values

### <a name='a4'></a> 5. Budowa modelu

Model1

In [28]:
def build_model1():
    model1 = Sequential()
    model1.add(Dense(1024, kernel_regularizer='l2', activation='relu', input_shape=[len(train_dataset.keys())]))
    model1.add(Dense(512, activation='relu'))
    model1.add(Dense(128, activation='relu'))
    model1.add(Dense(1)) #model regresyjny, na wyjściowej nie dajemy funkcji aktywacji



    model1.compile(optimizer='adam',
                  loss='mse',
                  metrics=['mae', 'mse'])
    return model1

In [30]:
model1 = build_model1()
model1.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 1024)           │        13,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 512)            │       524,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │        65,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 603,905 (2.30 MB)

 Trainable params: 603,905 (2.30 MB)

 Non-trainable params: 0 (0.00 B)

### <a name='a5'></a> 6. Trenowanie sieci

In [31]:
history = model1.fit(normed_train_data, train_labels.values, epochs=150, validation_split=0.2, verbose=1, batch_size=32)

Epoch 1/150
390/390 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - loss: 31969128448.0000 - mae: 145397.5156 - mse: 31969128448.0000 - val_loss: 4620013056.0000 - val_mae: 49139.3750 - val_mse: 4620013056.0000
Epoch 2/150
390/390 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 4443127808.0000 - mae: 47791.3477 - mse: 4443127808.0000 - val_loss: 3792327936.0000 - val_mae: 44711.2500 - val_mse: 3792327936.0000
Epoch 3/150
390/390 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 3803083776.0000 - mae: 44424.9727 - mse: 3803083776.0000 - val_loss: 3594418688.0000 - val_mae: 43840.1914 - val_mse: 3594418688.0000
Epoch 4/150
390/390 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 3589831936.0000 - mae: 43029.1406 - mse: 3589831936.0000 - val_loss: 3499535360.0000 - val_mae: 42769.1562 - val_mse: 3499535360.0000
Epoch 5/150
390/390 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 3459759872.0000 - mae: 42295.4688 - mse: 3459759872.0000 - val_loss: 3451603968.0000 - val_mae: 42859.7031 - val_mse: 3451603968.0000
Epoch 6/150
390/390 ━━

Po analizie widac, że można by dać regularyzacje

In [32]:
def plot_hist(history):
    hist = pd.DataFrame(history.history)
    hist['epoch'] = history.epoch
    hist['rmse'] = np.sqrt(hist['mse'])
    hist['val_rmse'] = np.sqrt(hist['val_mse'])

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['mae'], name='mae', mode='markers+lines'))
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['val_mae'], name='val_mae', mode='markers+lines'))
    fig.update_layout(width=1000, height=500, title='MAE vs. VAL_MAE', xaxis_title='Epoki', yaxis_title='Mean Absolute Error', yaxis_type='log')
    fig.show()

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['rmse'], name='rmse', mode='markers+lines'))
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['val_rmse'], name='val_rmse', mode='markers+lines'))
    fig.update_layout(width=1000, height=500, title='RMSE vs. VAL_RMSE', xaxis_title='Epoki', yaxis_title='Root Mean Squared Error', yaxis_type='log')
    fig.show()

plot_hist(history)

Metryka na zbiorze testowym

Porównać należy mae z testowego z mae z treningowego

In [33]:
for name, value in zip(model1.metrics_names, model1.evaluate(normed_test_data, test_labels.values)):
    print(f'{name:8}{value:.4f}')

122/122 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 2410372864.0000 - mae: 35930.3984 - mse: 2410372864.0000
loss    2408153600.0000
compile_metrics35688.5234


predict tagert na danych testowych

In [34]:
test_predictions = model1.predict(normed_test_data).flatten()
test_predictions

122/122 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step


array([236133.36, 209336.02, 185685.31, ..., 102526.22, 136420.22,  79479.4 ], dtype=float32)

In [35]:
pred = pd.DataFrame(test_labels)
pred['predictions'] = test_predictions
pred.head()

,median_house_value,predictions
10,281500.000000,236133.359375
13,191300.000000,209336.015625
19,162900.000000,185685.312500
26,93800.000000,157119.671875
40,155400.000000,183884.078125


Wykres Q-Q

In [36]:
fig = px.scatter(pred, 'median_house_value', 'predictions')
fig.add_trace(go.Scatter(x=[0, 500000], y=[0, 500000], mode='lines'))
fig.show()

In [37]:
pred.head()

,median_house_value,predictions
10,281500.000000,236133.359375
13,191300.000000,209336.015625
19,162900.000000,185685.312500
26,93800.000000,157119.671875
40,155400.000000,183884.078125


In [38]:
pred['error'] = pred['median_house_value'] - pred['predictions']
pred.head()

,median_house_value,predictions,error
10,281500.000000,236133.359375,45366.640625
13,191300.000000,209336.015625,-18036.015625
19,162900.000000,185685.312500,-22785.312500
26,93800.000000,157119.671875,-63319.671875
40,155400.000000,183884.078125,-28484.078125


In [39]:
px.histogram(pred, 'error', marginal='rug', width=1000)

MODEL 2

In [43]:
from tensorflow.keras import regularizers

def build_model2():
    model2 = Sequential()

    # Każda warstwa ma L2 regularizację
    model2.add(Dense(1024, activation='relu',
                     kernel_regularizer=regularizers.l2(0.001),
                     input_shape=[len(train_dataset.keys())]))

    model2.add(Dense(512, activation='relu',
                     kernel_regularizer=regularizers.l2(0.001)))

    model2.add(Dense(128, activation='relu',
                     kernel_regularizer=regularizers.l2(0.001)))

    # Wyjście modelu regresyjnego (bez aktywacji)
    model2.add(Dense(1))

    model2.compile(
        optimizer='adam',
        loss='mse',
        metrics=['mae', 'mse']
    )

    return model2


In [44]:
model2 = build_model1()
model2.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_4 (Dense)                 │ (None, 1024)           │        13,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 512)            │       524,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 128)            │        65,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 603,905 (2.30 MB)

 Trainable params: 603,905 (2.30 MB)

 Non-trainable params: 0 (0.00 B)

In [45]:
history2 = model2.fit(normed_train_data, train_labels.values, epochs=150, validation_split=0.2, verbose=1, batch_size=32)

Epoch 1/150
390/390 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - loss: 31349772288.0000 - mae: 144085.7344 - mse: 31349772288.0000 - val_loss: 4691016192.0000 - val_mae: 49628.9102 - val_mse: 4691016192.0000
Epoch 2/150
390/390 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 4233450496.0000 - mae: 46929.6836 - mse: 4233450496.0000 - val_loss: 3809469440.0000 - val_mae: 44719.6250 - val_mse: 3809469440.0000
Epoch 3/150
390/390 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 3721086720.0000 - mae: 44028.5078 - mse: 3721086720.0000 - val_loss: 3604040192.0000 - val_mae: 43779.0703 - val_mse: 3604040192.0000
Epoch 4/150
390/390 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 3636150272.0000 - mae: 43365.9336 - mse: 3636150272.0000 - val_loss: 3514381568.0000 - val_mae: 43277.5078 - val_mse: 3514381568.0000
Epoch 5/150
390/390 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 3623985920.0000 - mae: 43123.4883 - mse: 3623985920.0000 - val_loss: 3441785600.0000 - val_mae: 42331.5586 - val_mse: 3441785600.0000
Epoch 6/150
390/390 ━━

In [50]:
def plot_hist(history2):
    hist = pd.DataFrame(history2.history)
    hist['epoch'] = history2.epoch
    hist['rmse'] = np.sqrt(hist['mse'])
    hist['val_rmse'] = np.sqrt(hist['val_mse'])

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['mae'], name='mae', mode='markers+lines'))
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['val_mae'], name='val_mae', mode='markers+lines'))
    fig.update_layout(width=1000, height=500, title='MAE vs. VAL_MAE', xaxis_title='Epoki', yaxis_title='Mean Absolute Error', yaxis_type='log')
    fig.show()

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['rmse'], name='rmse', mode='markers+lines'))
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['val_rmse'], name='val_rmse', mode='markers+lines'))
    fig.update_layout(width=1000, height=500, title='RMSE vs. VAL_RMSE', xaxis_title='Epoki', yaxis_title='Root Mean Squared Error', yaxis_type='log')
    fig.show()

plot_hist(history)

In [51]:
for name, value in zip(model2.metrics_names, model2.evaluate(normed_test_data, test_labels.values)):
    print(f'{name:8}{value:.4f}')

122/122 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 2139764608.0000 - mae: 31737.7578 - mse: 2139764608.0000
loss    2232786944.0000
compile_metrics32460.2695


In [54]:
test_predictions2 = model2.predict(normed_test_data).flatten()
test_predictions2

122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step


array([224445.23, 187261.83, 169661.98, ...,  87644.37, 122015.71,  64416.65], dtype=float32)

In [55]:
pred2 = pd.DataFrame(test_labels)
pred2['predictions'] = test_predictions2
pred2.head()

,median_house_value,predictions
10,281500.000000,224445.234375
13,191300.000000,187261.828125
19,162900.000000,169661.984375
26,93800.000000,144832.671875
40,155400.000000,167585.265625


In [56]:
fig = px.scatter(pred2, 'median_house_value', 'predictions')
fig.add_trace(go.Scatter(x=[0, 500000], y=[0, 500000], mode='lines'))
fig.show()

In [57]:
pred2['error'] = pred2['median_house_value'] - pred2['predictions']
pred2.head()

,median_house_value,predictions,error
10,281500.000000,224445.234375,57054.765625
13,191300.000000,187261.828125,4038.171875
19,162900.000000,169661.984375,-6761.984375
26,93800.000000,144832.671875,-51032.671875
40,155400.000000,167585.265625,-12185.265625


In [58]:
px.histogram(pred2, 'error', marginal='rug', width=1000)

MODEL 3

In [67]:

def build_model3():
    model3 = Sequential()

    # Każda warstwa ma L2 regularizację
    model3.add(Dense(1024, activation='relu',
                     kernel_regularizer=regularizers.l2(0.001),
                     bias_regularizer=regularizers.l2(0.001),
                     activity_regularizer=regularizers.l2(0.001),
                     input_shape=[len(train_dataset.keys())]))

    model3.add(Dense(512, activation='relu',
                     kernel_regularizer=regularizers.l2(0.001),
                     bias_regularizer=regularizers.l2(0.001),
                     activity_regularizer=regularizers.l2(0.001)))

    model3.add(Dense(128, activation='relu',
                     kernel_regularizer=regularizers.l2(0.001),
                     bias_regularizer=regularizers.l2(0.001),
                     activity_regularizer=regularizers.l2(0.001)))

    # Wyjście modelu regresyjnego (bez aktywacji)
    model3.add(Dense(1))

    model3.compile(
        optimizer='adam',
        loss='mse',
        metrics=['mae', 'mse']
    )

    return model3

In [68]:
model3 = build_model3()
model3.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_8 (Dense)                 │ (None, 1024)           │        13,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 512)            │       524,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 128)            │        65,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 603,905 (2.30 MB)

 Trainable params: 603,905 (2.30 MB)

 Non-trainable params: 0 (0.00 B)

In [69]:
history3 = model3.fit(normed_train_data, train_labels.values, epochs=80, validation_split=0.2, verbose=1, batch_size=32)

Epoch 1/80
390/390 ━━━━━━━━━━━━━━━━━━━━ 6s 9ms/step - loss: 32369649664.0000 - mae: 145504.7812 - mse: 32313542656.0000 - val_loss: 4961807872.0000 - val_mae: 49894.4805 - val_mse: 4798062592.0000
Epoch 2/80
390/390 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 4437865472.0000 - mae: 47198.9453 - mse: 4265697792.0000 - val_loss: 3983145472.0000 - val_mae: 45265.5039 - val_mse: 3811410944.0000
Epoch 3/80
390/390 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 4138289152.0000 - mae: 45364.4414 - mse: 3970527232.0000 - val_loss: 3788623616.0000 - val_mae: 42887.8555 - val_mse: 3642417920.0000
Epoch 4/80
390/390 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 3846979328.0000 - mae: 43494.4688 - mse: 3695140352.0000 - val_loss: 3628063232.0000 - val_mae: 42650.8203 - val_mse: 3486983168.0000
Epoch 5/80
390/390 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 3710104064.0000 - mae: 42726.0273 - mse: 3568284160.0000 - val_loss: 3615753984.0000 - val_mae: 43350.0625 - val_mse: 3477454848.0000
Epoch 6/80
390/390 ━━━━━━━━

In [70]:
plot_hist(history3)

In [71]:
for name, value in zip(model3.metrics_names, model3.evaluate(normed_test_data, test_labels.values)):
    print(f'{name:8}{value:.4f}')

122/122 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 2255626752.0000 - mae: 33408.3672 - mse: 2238330624.0000
loss    2417891072.0000
compile_metrics34758.2773


In [72]:
test_predictions3 = model3.predict(normed_test_data).flatten()
pred3 = pd.DataFrame(test_labels)
pred3['predictions'] = test_predictions3
pred3['error'] = pred3['median_house_value'] - pred3['predictions']

display(pred3.head())
px.scatter(pred3, 'median_house_value', 'predictions').add_trace(go.Scatter(x=[0, 500000], y=[0, 500000], mode='lines')).show()
px.histogram(pred3, 'error', marginal='rug', width=1000).show()

122/122 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step


,median_house_value,predictions,error
10,281500.000000,221840.515625,59659.484375
13,191300.000000,184245.515625,7054.484375
19,162900.000000,159855.234375,3044.765625
26,93800.000000,139016.578125,-45216.578125
40,155400.000000,160671.937500,-5271.937500


Model 4 - z dropout

In [75]:
def build_model4():
    model4 = Sequential()

    # Każda warstwa ma L2 regularizację i dropout
    model4.add(Dense(1024, activation='relu',
                     kernel_regularizer=regularizers.l2(0.001),
                     bias_regularizer=regularizers.l2(0.001),
                     activity_regularizer=regularizers.l2(0.001),
                     input_shape=[len(train_dataset.keys())]))

    model4.add(Dropout(0.2))

    model4.add(Dense(512, activation='relu',
                     kernel_regularizer=regularizers.l2(0.001),
                     bias_regularizer=regularizers.l2(0.001),
                     activity_regularizer=regularizers.l2(0.001)))

    model4.add(Dropout(0.2))

    model4.add(Dense(128, activation='relu',
                     kernel_regularizer=regularizers.l2(0.001),
                     bias_regularizer=regularizers.l2(0.001),
                     activity_regularizer=regularizers.l2(0.001)))

    model4.add(Dropout(0.2))

    # Wyjście modelu regresyjnego (bez aktywacji)
    model4.add(Dense(1))

    model4.compile(
        optimizer='adam',
        loss='mse',
        metrics=['mae', 'mse']
    )

    return model4

In [77]:
model4 = build_model3()
model4.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_16 (Dense)                │ (None, 1024)           │        13,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_17 (Dense)                │ (None, 512)            │       524,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_18 (Dense)                │ (None, 128)            │        65,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_19 (Dense)                │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 603,905 (2.30 MB)

 Trainable params: 603,905 (2.30 MB)

 Non-trainable params: 0 (0.00 B)

In [78]:
history4 = model3.fit(normed_train_data, train_labels.values, epochs=90, validation_split=0.2, verbose=1, batch_size=32)

Epoch 1/90
390/390 ━━━━━━━━━━━━━━━━━━━━ 9s 11ms/step - loss: 31352049664.0000 - mae: 142899.6875 - mse: 31298676736.0000 - val_loss: 4706283520.0000 - val_mae: 48952.7656 - val_mse: 4540943872.0000
Epoch 2/90
390/390 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 4750647296.0000 - mae: 48562.7266 - mse: 4585731584.0000 - val_loss: 3964530944.0000 - val_mae: 45094.3789 - val_mse: 3803264512.0000
Epoch 3/90
390/390 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 4023367168.0000 - mae: 45134.6836 - mse: 3868469504.0000 - val_loss: 3775508992.0000 - val_mae: 43996.9102 - val_mse: 3628098304.0000
Epoch 4/90
390/390 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 3997500672.0000 - mae: 44687.0781 - mse: 3851729152.0000 - val_loss: 3643218176.0000 - val_mae: 43173.4844 - val_mse: 3503904768.0000
Epoch 5/90
390/390 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 3852832256.0000 - mae: 43730.1719 - mse: 3717841920.0000 - val_loss: 3599256576.0000 - val_mae: 42052.5938 - val_mse: 3477091840.0000
Epoch 6/90
390/390 ━━━━━━━

In [79]:
plot_hist(history4)

In [80]:
for name, value in zip(model4.metrics_names, model4.evaluate(normed_test_data, test_labels.values)):
    print(f'{name:8}{value:.4f}')

122/122 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 44308099072.0000 - mae: 187575.7031 - mse: 44308099072.0000
loss    46619107328.0000
compile_metrics193120.1875


In [81]:
test_predictions4 = model3.predict(normed_test_data).flatten()
pred4 = pd.DataFrame(test_labels)
pred4['predictions'] = test_predictions3
pred4['error'] = pred4['median_house_value'] - pred4['predictions']

display(pred4.head())
px.scatter(pred4, 'median_house_value', 'predictions').add_trace(go.Scatter(x=[0, 500000], y=[0, 500000], mode='lines')).show()
px.histogram(pred4, 'error', marginal='rug', width=1000).show()

122/122 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step


,median_house_value,predictions,error
10,281500.000000,221840.515625,59659.484375
13,191300.000000,184245.515625,7054.484375
19,162900.000000,159855.234375,3044.765625
26,93800.000000,139016.578125,-45216.578125
40,155400.000000,160671.937500,-5271.937500
